# Package Imports

In [1]:
import pandas as pd
import numpy as np

# Feature Engineering

In [2]:
# load data from csv
transactions_df = pd.read_csv('data/clean_transactions.csv')
transactions_df.head(2)

,accountNumber,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,...,accountOpenDate,dateOfLastAddressChange,cardCVV,enteredCVV,cardLast4Digits,transactionType,currentBalance,cardPresent,expirationDateKeyInMatch,isFraud
0,100088067,100088067,50000,49953.81,2016-04-08 19:56:36,200.12,1st BBQ,US,US,9.0,...,2014-01-09,2014-01-09,546,546,4615,PURCHASE,46.19,True,False,False
1,100088067,100088067,50000,49929.31,2016-11-09 07:26:06,95.11,1st Pub,US,US,2.0,...,2014-01-09,2014-01-09,402,402,2077,PURCHASE,70.69,True,False,False


### Add feature for purchase paired with a reversal.

In [3]:
# load reversal transaction pairs from txt
reversal_purchase_matches_idx = np.loadtxt('data/matching_purchases.txt', dtype=int)

# set pairedWithReversal to 1 if the transaction has is a purchase and matches a reversal
transactions_df['pairedWithReversal'] = transactions_df.index.isin(reversal_purchase_matches_idx).astype(int)

### Add feature for multiswipe transactions.

In [4]:
# load multiswipe transactions from txt
multiswipe_transaction_idx = np.loadtxt('data/multiswipe_transactions.txt', dtype=int)

# set isMultiswipe to 1 if the transaction is a multiswipe
transactions_df['isMultiswipe'] = transactions_df.index.isin(multiswipe_transaction_idx).astype(int)

### Add feature for whether enteredCVV and cardCVV are the same
- then drop enteredCVV and cardCVV

In [5]:
transactions_df['enteredAndCardCVVMatch'] = (transactions_df['enteredCVV'] == transactions_df['cardCVV']).astype(int)
transactions_df.drop(columns=['enteredCVV', 'cardCVV'], inplace=True)

### Combine accountNumber and customerId if they provide the same information

In [6]:
# check if accountNumber and customerId are ever different -> NO, ALWAYS THE SAME!
print(f"There {(transactions_df['accountNumber'] != transactions_df['customerId']).sum()} records where are accountNumber and customerId fields don't match!")

# drop customerId
transactions_df.drop(columns=['customerId'], inplace=True)

There 0 records where are accountNumber and customerId fields don't match!


### Convert time data to int values (ex. mins, hrs, days, etc.)

In [7]:
# convert dateOfLastAddressChange to daysSinceLastAddressChange
transactions_df['daysSinceLastAddressChange'] = (pd.to_datetime('today') - pd.to_datetime(transactions_df['dateOfLastAddressChange'])).dt.days

# convert accountOpenDate -> daysAccountAge
transactions_df['daysAccountAge'] = (pd.to_datetime('today') - pd.to_datetime(transactions_df['accountOpenDate'])).dt.days

# confirm calculations are correct
transactions_df[['dateOfLastAddressChange', 'daysSinceLastAddressChange', 'accountOpenDate', 'daysAccountAge']].head(2)

# drop columns
# Note: dropped currentExpDate because I would have used it for daysTillExpiry which would be inversly proportional to daysAccountAge (assuming credit cards are valid for set # of years when opened)
transactions_df.drop(columns=['dateOfLastAddressChange', 'accountOpenDate', 'currentExpDate'], inplace=True)

### Add feature for accountHistoryOfFraud
- if a customer has a history of fraud, they are more likely to commit fraud again
- if the data being provided to the model

In [8]:
# add accountHistoryOfFraud
transactions_df['accountHistoryOfFraud'] = transactions_df.groupby('accountNumber')['isFraud'].cumsum()
transactions_df['accountHistoryOfFraud'] = transactions_df['accountHistoryOfFraud'].apply(lambda x: 1 if x > 0 else 0)

### Add feature for time difference since the last transaction by the same customer.

In [9]:
# convert 'transactionsDateTime' to datetime object
transactions_df['transactionDateTime'] = pd.to_datetime(transactions_df['transactionDateTime'])

# sort by customerId and transactionDateTime
transactions_df.sort_values(['accountNumber', 'transactionDateTime'], inplace=True)

transactions_df['minsSinceLastTransaction'] = transactions_df.groupby('accountNumber')['transactionDateTime'].diff(1).dt.total_seconds() / 60

# confirm minsSinceLastTransation is only positive
print(f"There are {len(transactions_df[transactions_df['minsSinceLastTransaction'] < 0])} records with negative minsSinceLastTransaction!")

# drop datetime after using it to create features
transactions_df.drop(columns=['transactionDateTime'], inplace=True)

There are 0 records with negative minsSinceLastTransaction!


### Drop Merchant Name
- too many unique values
- if I were to use this, I would standardize all merchant names (ex. AMC #XXXX -> AMC, Uber #XXXX -> Uber, etc.)

In [10]:
# drop merchant name
transactions_df.drop(columns=['merchantName'], inplace=True)

### One-hot encode all categorical features.
- one hot encode acqCountry, merchantCountryCode, posEntryMode, posConditionCode, merchantCategoryCode, transactionType

In [11]:
# one hot encode all categorical features
categorical_features = ['acqCountry', 'merchantCountryCode', 'posEntryMode', 'posConditionCode', 'merchantCategoryCode', 'transactionType']

# one hot encode categorical features
one_hot_encoded_df = pd.get_dummies(transactions_df, columns=categorical_features, dtype=int)

### Convert boolean features to integer values 0 and 1.

In [12]:
# convert boolean values to 0/1
boolean_features = ['cardPresent', 'expirationDateKeyInMatch', 'isFraud']

one_hot_encoded_df[boolean_features] = one_hot_encoded_df[boolean_features].astype(int)

# Save Data For Modeling

In [13]:
# save data to csv
one_hot_encoded_df.to_csv('data/clean_engineered_with_outliers.csv', index=False)